# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [12]:
!pip install -q keras-tuner
!pip install -q datasets huggingface_hub

In [13]:
# !git clone https://github.com/thorsten789/hurricane_cloud_prediction.git

%cd /content/hurricane_cloud_prediction
!git pull

/content/hurricane_cloud_prediction
Already up to date.


In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers, models
from huggingface_hub import login, list_repo_files, hf_hub_download
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scr_model.Protokollierung import save_json_with_timestamp
from scr_model.losses import cubic_mse

## Model Selection

[Discuss the type(s) of models you consider for this task, and justify the selection.]



In [15]:
MODEL_NAME = "SimpleRNN"
# MODEL_NAME = "GRU"
# MODEL_NAME = "LSTM"

## Data Accessd

data access with huggingface 

In [ ]:
token = "hf_XXX"
REPO_ID = "mttfst/Paulette_Cloud_Tracks"

login(token)

In [23]:
# 1) Daten laden
files = list_repo_files(REPO_ID, repo_type="dataset")
files = [f for f in files if f.startswith("exp_1.1/") and f.endswith(".csv")]
print("Total CSV tracks:", len(files))

# 2) csv-Files mit mind. ca. 120 Zeitschritten (1 Stunde) rausfiltern

tracks_120 = pd.read_csv("/content/hurricane_cloud_prediction/5_Data_Sample/track_len.csv")

tracks_120 = tracks_120[tracks_120.track_len >= 120]
print("Total CSV tracks with at least 120 timesteps:",len(tracks_120))

tracks_120 = tracks_120.filename.to_list()

csv_files = [
    f for f in files
    if f.split("/")[1] in tracks_120
]
print(csv_files)

# 3) Reproduzierbar mischen
random.seed(42)        # fixer Seed, damit der Split immer gleich ist
csv_files_shuffled = csv_files.copy()
random.shuffle(csv_files_shuffled)

# 4) 70/15/15 Split auf Track-Ebene
n = len(csv_files_shuffled)
n_train = int(0.7 * n)
n_val   = int(0.15 * n)
# Rest geht in Test
n_test  = n - n_train - n_val

train_files = csv_files_shuffled[:n_train]
val_files   = csv_files_shuffled[n_train:n_train + n_val]
test_files  = csv_files_shuffled[n_train + n_val:]

print(f"Train tracks: {len(train_files)}")
print(f"Val tracks:   {len(val_files)}")
print(f"Test tracks:  {len(test_files)}")

# Optional: in einem Dict sammeln, damit es übersichtlich bleibt
split_files = {
    "train": train_files,
    "val": val_files,
    "test": test_files,
}

Total CSV tracks: 9227
Total CSV tracks with at least 120 timesteps: 1115
['exp_1.1/cell_00003.csv', 'exp_1.1/cell_00010.csv', 'exp_1.1/cell_00011.csv', 'exp_1.1/cell_00018.csv', 'exp_1.1/cell_00019.csv', 'exp_1.1/cell_00022.csv', 'exp_1.1/cell_00024.csv', 'exp_1.1/cell_00025.csv', 'exp_1.1/cell_00027.csv', 'exp_1.1/cell_00029.csv', 'exp_1.1/cell_00039.csv', 'exp_1.1/cell_00041.csv', 'exp_1.1/cell_00042.csv', 'exp_1.1/cell_00043.csv', 'exp_1.1/cell_00048.csv', 'exp_1.1/cell_00050.csv', 'exp_1.1/cell_00051.csv', 'exp_1.1/cell_00052.csv', 'exp_1.1/cell_00054.csv', 'exp_1.1/cell_00056.csv', 'exp_1.1/cell_00064.csv', 'exp_1.1/cell_00075.csv', 'exp_1.1/cell_00076.csv', 'exp_1.1/cell_00089.csv', 'exp_1.1/cell_00093.csv', 'exp_1.1/cell_00094.csv', 'exp_1.1/cell_00095.csv', 'exp_1.1/cell_00104.csv', 'exp_1.1/cell_00105.csv', 'exp_1.1/cell_00107.csv', 'exp_1.1/cell_00114.csv', 'exp_1.1/cell_00120.csv', 'exp_1.1/cell_00121.csv', 'exp_1.1/cell_00125.csv', 'exp_1.1/cell_00126.csv', 'exp_1.1/cell_0

In [18]:
def load_cloud_dataset(file_list, repo_id=REPO_ID):
    """
    Lädt eine Liste von Track-CSV-Dateien aus HF Dataset und gibt
    eine Liste von DataFrames zurück.
    """
    cloud_dfs = []

    for f in file_list:
        local_path = hf_hub_download(
            repo_id=repo_id,
            repo_type="dataset",
            filename=f
        )
        df = pd.read_csv(local_path)

        # nach Zeit sortieren
        if "time" in df.columns:
            df = df.sort_values("time").reset_index(drop=True)
        elif "frame" in df.columns:
            df = df.sort_values("frame").reset_index(drop=True)
        else:
            raise ValueError(f"No 'time' or 'frame' column in {f}")

        cloud_dfs.append(df)

    return cloud_dfs

In [24]:
train_dfs = load_cloud_dataset(train_files)
val_dfs   = load_cloud_dataset(val_files)
test_dfs  = load_cloud_dataset(test_files)

cell_08048.csv:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

cell_04650.csv:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

cell_08396.csv:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

cell_00104.csv:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

cell_05100.csv:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

...

cell_03316.csv:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

cell_03663.csv:   0%|          | 0.00/1.13M [00:00<?, ?B/s]

cell_04080.csv:   0%|          | 0.00/975k [00:00<?, ?B/s]

cell_00267.csv:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

cell_01579.csv:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

## Feature Engineering


We use the full vertical profiles (L00–L49) of microphysical and dynamic variables.
Non-physical metadata (e.g. time, location) are excluded from model inputs.

Then we create padding sequences.
The padding is needed because TensorFlow (and especially Keras) works internally with tensors of fixed shape. When training a batch of sequences, they must therefore be brought to the same length.

The sequences are then normalized.

### Target calculation

In [25]:
# calculate targets
def compute_targets(df: pd.DataFrame):
    df = df.sort_values("time").reset_index(drop=True)

    # Zeit in Minuten
    time = pd.to_datetime(df["time"])
    t_end = time.iloc[-1]

    remaining_lifetime = (
        (t_end - time).dt.total_seconds()/60).values

    return remaining_lifetime.reshape(-1, 1)

In [26]:
# define columns without physical data
META_COLS = [
    "frame", "time", "latitude", "longitude", "cell", "feature_orig"
]

def select_input_features(df: pd.DataFrame):
    return df.drop(columns=META_COLS)

### Padding
The padding is needed because TensorFlow (and especially Keras) works internally with tensors of fixed shape. When training a batch of sequences, they must therefore be brought to the same length.

In [39]:
PAD_TYPE = "post"
all_lengths = [len(df) for df in train_dfs + val_dfs + test_dfs]
MAX_LEN = max(all_lengths)
PAD_LENGTH = MAX_LEN # enspricht maximaler Tracklänge

def build_sequences(cloud_dfs, max_len=None):
    X, y = [], []

    for df in cloud_dfs:
        features = select_input_features(df).values
        targets = compute_targets(df)

        X.append(features)
        y.append(targets)

    X_pad = pad_sequences(X, padding=PAD_TYPE, dtype="float32", maxlen=PAD_LENGTH, value= -999)
    y_pad = pad_sequences(y, padding=PAD_TYPE, dtype="float32", maxlen=PAD_LENGTH, value= -999)

    return X_pad, y_pad

In [40]:
X_train, y_train = build_sequences(train_dfs)
X_val, y_val     = build_sequences(val_dfs)
X_test, y_test   = build_sequences(test_dfs)

### Normalization

In [41]:
def normalize_sequences(X):
    n_samples, timesteps, n_features = X.shape
    X_flat = X.reshape(-1, n_features)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_flat)

    return X_scaled.reshape(n_samples, timesteps, n_features), scaler

In [42]:
X_train, x_scaler = normalize_sequences(X_train)
X_val, _          = normalize_sequences(X_val)
X_test, _         = normalize_sequences(X_test)

## Hyperparameter Tuning

Random search with KerasTuner is used to explore:
- number of RNN units
- learning rate

Grid search is avoided due to computational cost.


In [43]:
def model_builder(hp):
    return build_rnn(
        input_shape=X_train.shape[1:],
        units1=hp.Int("units1", 32, 256, step=32),
        units2=hp.Int("units2", 32, 128, step=32),
        lr=hp.Float("lr", 1e-4, 1e-2, sampling="log")
    )

DO_TUNING = False  # Umschalter!

if DO_TUNING:
    tuner = kt.RandomSearch(
        model_builder,
        objective="val_loss",
        max_trials=20,
        directory="../tuning",
        project_name="simple_rnn"
    )

    tuner.search(X_train, y_train, validation_data=(X_val, y_val))
    best_hp = tuner.get_best_hyperparameters(1)[0]
else:
    best_hp = {
        "units1": 128,
        "units2": 64,
        "lr": 1e-3
    }

## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [44]:
def build_rnn(input_shape, units1=128, units2=64, lr=1e-3):
    inputs = layers.Input(shape=input_shape)

    x = layers.Masking(mask_value= -999.0)(inputs)
    x = layers.SimpleRNN(
        units1,
        activation="tanh",
        return_sequences=True
    )(x)

    x = layers.SimpleRNN(
        units2,
        activation="tanh",
        return_sequences=True
    )(x)

    outputs = layers.TimeDistributed(
        layers.Dense(1, activation="linear")
    )(x)

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=cubic_mse,
        metrics=[cubic_mse]
    )

    return model

In [45]:
EPOCHS = 50
BATCH_SIZE = 8

model = build_rnn(
    input_shape=X_train.shape[1:],
    units1=best_hp["units1"],
    units2=best_hp["units2"],
    lr=best_hp["lr"]
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 93s 947ms/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 87s 892ms/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 149s 960ms/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan
Epoch 5/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan
Epoch 6/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 134s 929ms/step - cubic_mse: nan - loss: nan - val_cubic_mse: nan - val_loss: nan


## Evaluation Metrics

We use a cubic error metric to penalize large prediction errors more strongly
than the classical MSE.


In [ ]:
val_loss = min(history.history["val_loss"])
test_loss = model.evaluate(X_test, y_test)

## Experiment Logging
Save (Hyper-)Parameters in json-file 

In [ ]:
experiment_data = {
    "model": MODEL_NAME,
    "parameters": {
        "padding_type": PAD_TYPE,
        "padding_length": PAD_LENGTH
    },
    "hyperparameters": best_hp,
    "training": {
        "epochs": len(history.history["loss"]),
        "batch_size": BATCH_SIZE,
        "early_stopping": True
    },
    "results": {
        "best_val_loss": float(val_loss),
        "test_loss": float(test_loss)
    }
}

save_json_with_timestamp("simple_rnn_experiment", experiment_data)


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]


In [ ]:
# Comparative Analysis code (if applicable)
# Example: comparing accuracy of the baseline model and the new model
# print(f"Baseline Model Accuracy: {baseline_accuracy}, New Model Accuracy: {new_model_accuracy}")
